In [2]:
import random
import pandas as pd
import numpy as np
import os
from glob import glob 
import cv2
import argparse



In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


: 

: 

In [1]:
import matplotlib.pyplot as plt

from tqdm import tqdm

import warnings

In [ ]:
USE_CUDA = True if torch.cuda.is_available() else False
DEVICE = 'cuda' if USE_CUDA else 'cpu'

parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=1)
parser.add_argument('--lr',type=float, default=1e-3)
parser.add_argument('--seed', type=int, default=99)
parser.add_argument('--L1_lambda', type=float, default=100.0)
parser.add_argument('--b1', type=float, default=0.5)
parser.add_argument('--b2', type=float, default=0.999)
args = parser.parse_args('')

In [ ]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = True

seed_everything(args.seed)

In [ ]:
class CBR2d(nn.Module):
  def __init__(self,
    in_channel,
    out_channel,
    kernel_size = 3,
    stride = 1,
    padding = 0,
    bias = False,
    norm = True
  ):
    super().__init__()

    layer = [
      nn.Conv2d(
        in_channels=in_channel,
        out_channels=out_channel,
        kernel_size=kernel_size,
        padding=padding,
        stride=stride,
        bias=bias
      )
    ]

    if norm:
      layer.append(nn.BatchNorm2d(out_channel))
    
    layer.append(nn.LeakyReLU(0.2, inplace=True))

    self.enc = nn.Sequential(*layer)

  def forward(self, x):
    return self.enc(x)


class DECBR2d(nn.Module):
  def __init__(self,
    in_channel,
    out_channel,
    kernel_size = 3,
    stride = 1,
    padding = 0,
    bias = False,
    norm = True,
    drop = False
  ):
    super().__init__()

    self.norm = norm
    self.drop = drop

    self.upsample = nn.ConvTranspose2d(
      in_channels=in_channel,
      out_channels=out_channel,
      kernel_size=kernel_size,
      padding=padding,
      stride=stride,
      bias=bias
    ) 
    
    self.batchnorm = nn.BatchNorm2d(out_channel)
    self.relu = nn.LeakyReLU(0.2, inplace=True)
    self.dropout = nn.Dropout(0.5)
  
  def forward(self, x):
    x = self.upsample(x)
    
    if self.norm:
      x = self.batchnorm(x)
    
    if self.drop:
      x = self.dropout(x)

    x = self.relu(x)

    return x

In [68]:
# Generate Segmentation map
def Generate_background_segmentation_map(root_resource_path):
    sem_image_path_list = sorted(glob(os.path.join(root_resource_path, 'SEM/*/*/*')))
    depth_image_path_list = sorted(glob(os.path.join(root_resource_path, 'Depth/*/*/*')) + glob(os.path.join(root_resource_path, 'Depth/*/*/*')))
    # segment_image_path_list = list()
    for depth_image_path in depth_image_path_list:
        image = cv2.imread(depth_image_path, cv2.IMREAD_GRAYSCALE)
        after_root_path = '/'.join(depth_image_path.split('/')[-3:])
        backbround_depth = image[0][0]
        segment_image = (image == backbround_depth).astype(np.int32) * 255
        # print(os.path.join(root_resource_path, 'segment' ,after_root_path))
        os.makedirs('/'.join(os.path.join(root_resource_path, 'Segment' ,after_root_path).split('/')[:-1]), exist_ok=True)
        cv2.imwrite(os.path.join(root_resource_path, 'Segment' ,after_root_path), segment_image)

Generate_background_segmentation_map('/Users/choihanjun/Downloads/open/simulation_data/')

In [2]:
simulation_sem_paths = sorted(glob('/Users/choihanjun/Downloads/open/simulation_data/SEM/*/*/*.png'))
simulation_depth_paths = sorted(glob('/Users/choihanjun/Downloads/open/simulation_data/Depth/*/*/*.png')+glob('/Users/choihanjun/Downloads/open/simulation_data/Depth/*/*/*.png'))
simulation_segment_paths = sorted(glob('/Users/choihanjun/Downloads/open/simulation_data/Segment/*/*/*.png') + glob('/Users/choihanjun/Downloads/open/simulation_data/Segment/*/*/*.png'))

print(len(simulation_sem_paths))
print(len(simulation_depth_paths))
print(len(simulation_segment_paths))

NameError: name 'glob' is not defined

In [60]:
class CustomDataset(Dataset):
  def __init__(self, sem_path_list, depth_path_list, transform=None):
    self.sem_path_list = sem_path_list
    self.depth_path_list = depth_path_list
    self.transform = transform
      
  def __getitem__(self, index):
    sem_path = self.sem_path_list[index]
    sem_img = cv2.imread(sem_path, cv2.IMREAD_GRAYSCALE)
    sem_img = np.expand_dims(sem_img, axis=-1).transpose(2,0,1)
    sem_img = sem_img / 255.
    
    if self.depth_path_list is not None:
      depth_path = self.depth_path_list[index]
      depth_img = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)
      depth_img = np.expand_dims(depth_img, axis=-1).transpose(2,0,1)
      depth_img = depth_img / 255.
      return torch.Tensor(sem_img), torch.Tensor(depth_img) # B,C,H,W
    else:
      img_name = sem_path.split('/')[-1]
      return torch.Tensor(sem_img), img_name # B,C,H,W
      
  def __len__(self):
    return len(self.sem_path_list)

In [3]:
data_len = len(simulation_sem_paths)

NameError: name 'simulation_sem_paths' is not defined

In [ ]:
train_sem_paths = simulation_sem_paths[:int(data_len*0.8)]
train_depth_paths = simulation_depth_paths[:int(data_len*0.8)]
train_segment_paths = simulation_segment_paths[:int(data_len*0.8)]

val_sem_paths = simulation_sem_paths[int(data_len*0.8):]
val_depth_paths = simulation_depth_paths[int(data_len*0.8):]
val_segment_paths = simulation_segment_paths[int(data_len*0.8):]

In [61]:
# train_dataset = CustomDataset(simulation_sem_paths, simulation_depth_paths)
train_dataset = CustomDataset(simulation_sem_paths, simulation_segment_paths)
train_loader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_sem_paths, val_depth_paths)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

In [62]:
class Unet(nn.Module):
  def __init__(self):
    super().__init__()

    self.down1 = CBR2d(in_channel=1, out_channel=64) # (B 70 46 64)
    self.down2 = CBR2d(in_channel=64, out_channel=128) # (B 68 44 128)
    self.down3 = CBR2d(in_channel=128, out_channel=256) # (B 66 42 256)
    self.down4 = CBR2d(in_channel=256, out_channel=512) # (B 64 40 512)
    self.down5 = CBR2d(in_channel=512, out_channel=512) # (B 62 38 512)
    self.down6 = CBR2d(in_channel=512, out_channel=512) # (B 60 36 512)
    self.down7 = CBR2d(in_channel=512, out_channel=512) # (B 58 34 512)
    self.down8 = CBR2d(in_channel=512, out_channel=512) # (B 56 32 512) # batch_size가 1이면 norm적용 불가능

    self.up8 = DECBR2d(in_channel=512, out_channel=512)
    self.up7 = DECBR2d(in_channel=1024, out_channel=512)
    self.up6 = DECBR2d(in_channel=1024, out_channel=512)
    self.up5 = DECBR2d(in_channel=1024, out_channel=512)
    self.up4 = DECBR2d(in_channel=1024, out_channel=256) 
    self.up3 = DECBR2d(in_channel=512, out_channel=128)
    self.up2 = DECBR2d(in_channel=256, out_channel=64)
    self.up1 = nn.ConvTranspose2d(
      in_channels=128,
      out_channels=1,
      kernel_size=3,
      stride=1,
      padding=0 
    )


  def forward(self, x):
    down1 = self.down1(x)
    down2 = self.down2(down1)
    down3 = self.down3(down2)
    down4 = self.down4(down3)
    down5 = self.down5(down4)
    down6 = self.down6(down5)
    down7 = self.down7(down6)
    down8 = self.down8(down7)

    up8 = self.up8(down8)
    cat7 = torch.cat((up8, down7), dim=1)
    up7 = self.up7(cat7)
    cat6 = torch.cat((up7, down6), dim=1)
    up6 = self.up6(cat6)
    cat5 = torch.cat((up6, down5), dim=1)
    up5 = self.up5(cat5)
    cat4 = torch.cat((up5, down4), dim=1)
    up4 = self.up4(cat4)
    cat3 = torch.cat((up4, down3), dim=1)
    up3 = self.up3(cat3)
    cat2 = torch.cat((up3, down2), dim=1)
    up2 = self.up2(cat2)
    cat1 = torch.cat((up2, down1), dim=1)
    up1 = self.up1(cat1)

    return up1

In [63]:
# dice_loss implement
def dice_loss(pred, target, smooth = 1e-5):
    # binary cross entropy loss
    bce = F.binary_cross_entropy_with_logits(pred, target, reduction='sum')

    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum(dim=(2,3))
    union = pred.sum(dim=(2,3)) + target.sum(dim = (2,3))

    # dice coefficient
    dice = 2.0 * (intersection + smooth) / (union + smooth)

    # dice loss
    dice_loss = 1.0 - dice

    # total loss
    loss = bce + dice_loss

    return loss.sum(), dice.sum()

In [64]:
from torchsummary import summary

model = Unet().cuda()
summary(model, input_size=(1, 72, 48))

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    rmse = nn.MSELoss().to(device)
    
    val_loss = []
    val_rmse = []
    with torch.no_grad():
        for sem, depth in tqdm(iter(val_loader)):
            sem = sem.float().to(device)
            depth = depth.float().to(device)
            
            model_pred = model(sem)
            loss = criterion(model_pred, depth)
            
            pred = (model_pred*255.).type(torch.int8).float()
            true = (depth*255.).type(torch.int8).float()
            
            b_rmse = torch.sqrt(criterion(pred, true))
            
            val_loss.append(loss.item())
            val_rmse.append(b_rmse.item())

    return np.mean(val_loss), np.mean(val_rmse)

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion_list = [nn.L1Loss().to(device), dice_loss]
    best_score = 999999
    best_model = None
    
    for epoch in range(1, args.epochs+1):
        model.train()
        train_loss = []
        for sem, depth in tqdm(iter(train_loader)):
            sem = sem.float().to(device)
            depth = depth.float().to(device)
            
            optimizer.zero_grad()
            
            model_pred = model(sem)
            loss = criterion(model_pred, depth)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        val_loss, val_rmse = validation(model, criterion, val_loader, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}] Val RMSE : [{val_rmse:.5f}]')
        
        if best_score > val_rmse:
            best_score = val_rmse
            best_model = model
        
        if scheduler is not None:
            scheduler.step()
            
    return best_model

In [ ]:
model = Unet()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = args.lr)
scheduler = None

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, DEVICE)